![Example12_9.png](Example12_9.png)
![Example12_9-1.png](Example12_9-1.png)

In [1]:
import CompositeBeam as comp

### Variables

In [2]:
# Yapım aşaması yükleri
DeadLoad   = 2.5 #kN/m^2
LiveLoad_c = 1 #kN/m^2

# Kompozit çalışma aşaması yükleri
SuperDeadLoad = 2.0 #kN/m^2
LiveLoad      = 4.0 #kN/m^2

# Beton bilgisi
f_ck     = 30     #N/mm2

# Çelik malzeme bilgisi
Fy       = 355    #N/mm2
Fu_beam  = 450    #N/mm2

# Çelik enkesit özellikleri
Abeam    = 9_880  #mm^2
Hbeam    = 450    #mm
Hweb     = 378.8  #mm
b_flange = 190    #mm
t_flange = 14.6   #mm
t_web    = 9.4    #mm
Ix       = 33_740*10**4 #mm^4
Ycon     = 130    #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

# Kiriş uzunluk ve aralık bilgisi
Laralık  = 3000   #mm
Lbeam    = 7500  #mm

# Stud çivisi bilgisi
D_stud   = 19
H_stud   = 100
Fu_stud  = 450 #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı

# Metal sac bilgisi
hr       = 50     #mm
wr       = 165    #mm
t_sac    = 1.2    #mm


### Design Code Requirements

In [3]:
comp.ConcAvailableStressCheck(f_ck=f_ck)
comp.SteelAvailableStressCheck(Fy=Fy)
comp.MetalDeckCheck(hr=hr,wr=wr)
comp.ConcreteCoverAboveTopOfHeadedStudAnchorsChecks(hr=hr,t_sac=t_sac, h_stud=H_stud, Ycon=Ycon)
comp.StudCheck(Ds=D_stud, tf=t_flange, Hs=H_stud, IsWebAlignmentWelded=False)

20 N/mm^2 ≤ 30 N/mm^2 < 70 N/mm^2 √ - TurkishSteelStructureCode 12.2.3(a)
355 N/mm^2 ≤ 460 N/mm^2 √ - TurkishSteelStructureCode 12.2.3(c)
hr = 50mm ≤ 75mm √ - TurkishSteelStructureCode 12.4.2.3
wr = 165mm ≥ 50mm √ - TurkishSteelStructureCode 12.4.2.3
h_stud-hr-t_sac = 48.8mm ≥ 38mm √ - TurkishSteelStructureCode 12.4.2.3
Ycon - h_stud-t_sac = 28.8mm ≥ 13mm √ - TurkishSteelStructureCode 12.4.2.3
Ycon - hr = 80mm ≥ 50mm √ - TurkishSteelStructureCode 12.4.2.3
Ds = 19mm ≤ 2.5*tf = 36.5mm √ - TurkishSteelStructureCode 
Ds = 19mm ≤ 19mm √ - TurkishSteelStructureCode 12.8.1
Hs = 100mm ≥ 4*Ds = 76mm √ - TurkishSteelStructureCode 12.8.2


True

### Yapım aşaması deplasman kontrolü

In [4]:
# Yapım aşaması
w_girder = 1.0 #kN/m kiriş ağırlığı
W_G = comp.calc_w_cdl(DeadLoad=DeadLoad, GirdersSpaceLength=Laralık/10**3) + w_girder
W_Q = comp.calc_w_ll(LiveLoad=LiveLoad_c, GirdersSpaceLength=Laralık/10**3)
W_G,W_Q

(8.5, 3.0)

In [5]:
# Yapım aşaması sabit yüklerinden(döşeme ve çelik kiriş) kaynaklı deplasman
delta_cdl = comp.SimpleCompositeBeamDeflection(w=W_G, Lbeam=Lbeam, I=Ix)
delta_cdl

5.19

In [6]:
print(f"{delta_cdl} < {(Lbeam/360)}")

5.19 < 20.833333333333332


In [7]:
# Ters sehim miktarı
delta_c = comp.Camber(delta_cdl=delta_cdl, Lbeam=Lbeam)

delta_cdl = 5.19mm < 19mm or Lbeam = 7500 < 7600mm olduğu için ters sehime gerek yoktur.


In [8]:
# Yapım aşaması hareketli yüklerinden kaynaklı deplasman
delta_yll = comp.SimpleCompositeBeamDeflection(w=W_Q, Lbeam=Lbeam, I=Ix)
delta_yll

1.83

In [9]:
# Yapım aşaması toplam düşey yerdeğiştirme, super dead düşey yerdeğiştirmesi yapım öncesi olmadığı için 0.0
delta_ytl = comp.calc_Delta_TL(delta_cdl=delta_cdl, delta_sdl=0., delta_ll=delta_yll,delta_c=delta_c)
delta_ytl

7.02

### Kompozit kirişin karakteristik eğilme momenti dayanımının belirlenmesi

In [10]:
# Kirişenkesit kontrolü
alfa = Hbeam/t_web
print(f"alfa = {round(alfa,2)} ≤ 3.76*(E/Fy)^0.5 = {round(3.76*(2*10**5/Fy)**0.5,2)} √")

alfa = 47.87 ≤ 3.76*(E/Fy)^0.5 = 89.25 √


In [11]:
# Effektif döşeme genişliği
b_eff = comp.EffectiveSlabWidth(L=Lbeam, Lu=Laralık)
b_eff

1875.0

In [12]:
# Stud çivisiz beton yüksekliği
tc = comp.Calc_tc(hr=hr,Ycon=Ycon,IsParallel=False)
tc

80

In [13]:
# Stud çivisiz beton alanı
Ac = b_eff * tc
Ac

150000.0

In [14]:
Cs = comp.YieldBeamCapacity(fy=Fy,As=Abeam)
Cs

3507400

In [15]:
Cc = comp.CrushConcCapacity(fck=f_ck,Ac=Ac)
Cc

3825000.0

In [16]:
Mn_design = comp.CompositeBeamDesignFlexuralCapacity(C_conc=Cc, T_steel=Cs, Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac = Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
Mn_design/10**6

a = 73.35738562091504 mm
Y2 = 93.32130718954248 mm
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.
φMn = φ * (min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a)) = 0.9 * 3507400 *  318.32 = 1004832137.553


1004.83213755

### Arttırılmış düşey yükler altında eğilme momenti kapasite talebi

In [17]:
W_superdeadload = comp.calc_w_sdl(SuperDeadLoad=SuperDeadLoad,GirdersSpaceLength=Laralık/10**3)
W_superdeadload

6.0

In [18]:
W_G_composite = W_G + W_superdeadload
W_G_composite

14.5

In [19]:
W_Q_composite = comp.calc_w_ll(LiveLoad=LiveLoad, GirdersSpaceLength=Laralık/10**3)
W_Q_composite

12.0

In [20]:
W_TL = comp.calc_w_tl(w_dl=W_G_composite, w_ll=W_Q_composite)
W_TL

36.6

In [21]:
M_demand = W_TL * Lbeam**2 /8
M_demand/10**6

257.34375

### Talep/Kapasite oranı

In [22]:
flexuralcapacityratio = round(M_demand / Mn_design,2)
flexuralcapacityratio

0.26

### Başlıklı çelik ankrajların sayısı ve yerleşimi

In [23]:
Ec = comp.ConcreteYoungModules(fck=f_ck, wc=2400)
Ec

27691.465833357393

In [24]:
A_sa = 3.14 * D_stud**2 /4
A_sa

283.385

In [25]:
Rg = comp.GetRg(StudsNumberInFlangeWidth=2, wr=wr, hr=hr, IsPitchParaleltoBeam=False)
Rg

0.85

In [26]:
Rp = comp.GetRp(Hs = H_stud, hr =hr, t_studhead = 5, IsPitchParaleltoBeam=False)
Rp

0.75

In [27]:
Qn = comp.OneStudShearCapacity(Asa=A_sa, fck=f_ck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=Fu_beam)
Qn

Qn = 0.5 * Asa * (fck*Ec)**0.5 = 129145.852
Qn = 92135.548 > Rg*Rp*Asa*Fu = 129145.852 ==> Qn = 129145.852


92135.548

In [28]:
# TotalQn = ShearStudCapacity(N=N, Qn=Qn)
# Itr = Calc_I_tr(n=n, b_eff=beff,hr=hr,hc=hc,hb=Hbeam,Ab=BeamAs,Ibeam=Ibeam)
# Cf = Calc_Cf(Vbeam=T_steel,Vcon=C_conc)
# Ieff = Calc_I_eff(Ibeam=Ibeam, TotalQn=TotalQn, Cf=Cf, Itr=Itr)